# Guided Project: Building a database for crime reports


In this guided project, we will put everything together to build a database for storing data related with crimes that occurred in Boston. This dataset is available in the file boston.csv. The first four rows are shown here:

|incident_number|offense_code|description|date|day_of_the_week|lat|long|
|---|---|---|---|---|---|---|
|1|619|LARCENY ALL OTHERS|2018-09-02|Sunday|42.35779134|-71.13937053
|2|1402|VANDALISM|2018-08-21|Tuesday|42.30682138|-71.06030035
|3|3410|TOWED MOTOR VEHICLE|2018-09-03|Monday|42.34658879|-71.07242943
|4|3114|INVESTIGATE PROPERTY|2018-09-03|Monday|42.33418175|-71.07866441

The first column represents the identifier of the crime. The second contains a numeric identifier code for the committed crime. The third represents a description of the crime. The next two rows contain the date on which the crime happened and the corresponding day of the week. Finally, the last two columns represent the location of the crime with a latitude and longitude coordinates.

The goal of this guided project is to create a database named crimes_db with a table – boston_crimes – with appropriate datatypes for storing the data from the boston.csv file. We will be creating the table inside a schema named crimes. We will also create the readonly and readwrite groups with the appropriate privileges. Finally, we will also need to create one user for each of these groups.

## Create the database and schema
Create a database named crime_db and a schema named crimes for storing the tables for containing the crime data.

We first need to connect to the "dq" database to create "crime_db" so we can then connect to it.

In [1]:
import psycopg2
conn = psycopg2.connect(dbname="dq", user="dq")
# Autocommit set to True, required for creating databases
conn.autocommit = True
cur = conn.cursor()
# create the crime_db database
cur.execute("CREATE DATABASE crime_db;")
conn.close()

ProgrammingError: database "crime_db" already exists


In [2]:
# After the crime_db is created we can now connect to it
conn = psycopg2.connect(dbname="crime_db", user="dq")
conn.autocommit = True
cur = conn.cursor()
# create schema named crimes
cur.execute("CREATE SCHEMA crimes;")

ProgrammingError: schema "crimes" already exists


## Collect the column names and first row
Gather the header row and assign it to "col_headers", then the first row of data to "first_row".

In [3]:
import csv
with open('boston.csv') as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)

## Create a function to analyze column values
Create a function "get_col_set" that collects all distinct values from a given CSV file name and a column index.

This function will be useful for two reasons:

- Checking whether an enumerated datatype might be a good choice for representing a column.
- Computing the maximum length of any text-like column to select appropriate sizes for VARCHAR columns.

Use the function on each column to evaluate which columns have many different values. Columns with a small set of possible values could possibly be enumerated datatypes.

In [4]:
def get_col_set(csv_file, col_index):
    import csv
    values = set()
    with open(csv_file, 'r') as f:
        next(f)
        reader = csv.reader(f)
        for row in reader:
            values.add(row[col_index])
    return values

for i in range(len(col_headers)):
    values = get_col_set("boston.csv", i)
    print(col_headers[i], len(values), sep=' = ')

incident_number = 298329
offense_code = 219
description = 239
date = 1177
day_of_the_week = 7
lat = 18177
long = 18177


## Analyze the maximum length of the description column

We can use the "get_col_set" function for computing this:

- Use the get_col_value_set() function to get the distinct values of that column into a variable named col_values
- Iterate over the elements of col_values and find the one which is the longest

In [5]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [6]:
descriptions = get_col_set("boston.csv", 2) # Index num 2 is description
max_len = 0
for description in descriptions:
    max_len = max(max_len, len(description))
print(max_len)

58


## Create the table
We can create an enumerated datatype named "weekday" for the day_of_the_week since there there only seven possible values.

For the incident_number we can set the datatype to INTEGER and set it as the primary key. The same datatype was also used to represent the offense_code.

Since the description has at most 58 character we decided to use the datatype VARCHAR(100) for representing it. This will leave some margin, while not being so big that we will waste a lot of memory.

The date can be represented as the DATE datatype. Finally, for the latitude and longitude we can use DECIMAL datatypes.

In [7]:
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


In [8]:
# Create the enumerated datatype for representing the weekday
cur.execute("""
    CREATE TYPE weekday AS ENUM ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');
""")

In [9]:
# Create the table
cur.execute("""
    CREATE TABLE crimes.boston_crimes (
        incident_number INTEGER PRIMARY KEY,
        offense_code INTEGER,
        description VARCHAR(100),
        date DATE,
        day_of_the_week weekday,
        lat decimal,
        long decimal
    );
""")

## Load the data into the table
Use the "copy_expert()" method to load the data.

In [10]:
# load the data from boston.csv into the table boston_crimes
# that is in the crimes schema
with open("boston.csv") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)
cur.execute("SELECT * FROM crimes.boston_crimes")

# Print the number of rows to check that they were loaded correctly
print(len(cur.fetchall()))

298329


## Revoke public privileges
We revoke all privileges of the public group on the public schema to ensure that users will not inherit privileges on that schema such as the ability to create tables in the public schema.

We also need to revoke all privileges in the newly created schema. Doing this also makes it so that we do not need to revoke the privileges when we create users and groups because unless specified otherwise, privileges are not granted by default.

In [11]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

## Create a read only group
We create a readonly group with NOLOGIN because it is a group and not a user. We grant the group the ability to connect to the crime_db and the ability to use the crimes schema.

Then we deal with tables privileges by granting SELECT. We also add an extra line compared with what was asked. This extra line changes the way that privileges are given by default to the readonly group on the new table that was created on the crimes schema. As we mentioned, by default not privileges are given. However we change is so that by default any user in the readonly group can issue select commands.

In [12]:
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")

## Create a read-write group
We create a readwrite group with NOLOGIN because it is a group and not a user. We grant the group the ability to connect to the crime_db and the ability to use the crimes schema.

Then we deal with tables privileges by granting SELECT, INSERT, UPDATE and DELETE. As before we change the default privileges so that users in the readwrite group have these privileges if we ever create a new table on the crimes schema.

In [13]:
cur.execute("CREATE GROUP readwrite NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")
cur.execute("GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA crimes TO readwrite;")

## Create one user for each group
We create a user named data_analyst with password secret1 in the readonly group.

We create a user named data_scientist with password secret2 in the readwrite group.

In [14]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("GRANT readonly TO data_analyst;")

cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readwrite TO data_scientist;")

## Test the database setup
Test the database setup using SQL queries on the pg_roles table and information_schema.table_privileges.

In the pg_roles table we will check database related privileges and for that we will look at the following columns:

- rolname: The name of the user / group that the privilege refers to.
- rolsuper: Whether this user / group is a super user. It should be set to False on every user / group that we have created.
- rolcreaterole: Whether user / group can create users, groups or roles. It should be False on every user / group that we have created.
- rolcreatedb: Whether user / group can create databases. It should be False on every user / group that we have created.
- rolcanlogin: Whether user / group can login. It should be True on the users and False on the groups that we have created.
In the information_schema.table_privileges we will check privileges related to SQL queries on tables. We will list the privileges of each group that we have created.

In [15]:
# Close the previous connection to test it with a brand new connection
conn.close()

conn = psycopg2.connect(dbname="crime_db", user="dq")
cur = conn.cursor()
# Check the users and groups
cur.execute("""
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
""")
for user in cur:
    print(user)
print()
# Check the privileges
cur.execute("""
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite');
""")
for user in cur:
    print(user)
conn.close()

('readonly', False, False, False, False)
('readwrite', False, False, False, False)
('data_analyst', False, False, False, True)
('data_scientist', False, False, False, True)

('readonly', 'SELECT')
('readwrite', 'INSERT')
('readwrite', 'SELECT')
('readwrite', 'UPDATE')
('readwrite', 'DELETE')
